In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pprint as pp
from pathlib import Path

sys.path.append(os.path.abspath(".."))

from utils.model_transformer.model import Model
from utils.model_ann.ann import build_index, search

device = "cpu"

# Load Dataset

In [2]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])

In [3]:
dataset_dir = Path(base_dir).joinpath(f"data/dataset/{DOMAIN}/interactions")
df = pd.read_parquet(dataset_dir)
df.head()

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3092,2025-09-21 03:01:00,click,46,F,"트렌디한 스포티룩, 세미오버핏 레깅스 in 여름",아이보리,스포티,세미오버핏,울,여름,7부,레깅스
1,1,8879,2025-09-21 03:02:00,click,46,F,"트렌디한 스포티룩, 슬림핏 레깅스 in 여름",아이보리,스포티,슬림핏,메쉬,여름,롱슬리브,레깅스
2,1,1880,2025-09-21 03:03:00,click,46,F,필수템! 스포티 무드의 간절기용 레깅스,아이보리,스포티,세미오버핏,데님,간절기,숏슬리브,레깅스
3,1,6154,2025-09-21 03:13:00,click,46,F,여름 감성 슬림핏 폴리 레깅스,아이보리,스포티,슬림핏,폴리,여름,롱슬리브,레깅스
4,1,8642,2025-09-21 03:13:00,wishlist,46,F,"트렌디한 스포티룩, 레귤러핏 레깅스 in 겨울",아이보리,스포티,레귤러핏,코튼,겨울,7부,레깅스


# Load Model

In [4]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])
model_dir = Path(base_dir).joinpath(f"data/model/{DOMAIN}")

In [5]:
model = Model(model_dir=model_dir)

/Users/hwangyungwan/miniforge3/envs/ds_p313/lib/python3.13/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## Prediction Test

In [6]:
body = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
        ],
    }
]

pp.pprint(model.predict(body))

[{'item_vector': [0.007925938814878464,
                  0.01774013042449951,
                  -0.01827966794371605,
                  0.0219673253595829,
                  0.021292395889759064,
                  -0.024046875536441803,
                  -0.022494960576295853,
                  -0.04081568866968155,
                  0.023688748478889465,
                  0.029071614146232605,
                  -0.03069879114627838,
                  -0.006091278046369553,
                  0.17021100223064423,
                  0.02889350801706314,
                  -0.03895937278866768,
                  -0.029103927314281464,
                  0.004552461206912994,
                  -0.02538055181503296,
                  0.07599465548992157,
                  0.008912317454814911,
                  0.041384898126125336,
                  -0.012605756521224976,
                  0.05165726691484451,
                  -0.026099994778633118,
                  0.02419680915772915,
  

# Vector Similarity Search

## Recommendation

In [7]:
item_metadata_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/item_metadata.parquet"
)
df_item = pd.read_parquet(item_metadata_path)

text_vectors_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/text_vectors.parquet"
)
df_text_vectors = pd.read_parquet(text_vectors_path)

In [8]:
text_vectors = np.array(df_text_vectors["text_vector"].tolist())
item_vectors = np.array(text_vectors).astype(np.float32)
item_id_maps = dict(zip(df_text_vectors["item_id"].index, df_text_vectors["item_id"]))

In [9]:
# item_vectors: (num_items, dim) numpy float32 배열이라고 가정
item_index = build_index(item_vectors)
item_idx = 0
query_vector = item_vectors[item_idx]
result = search(query_vector, item_index, top_k=5)
print(result)  # 예: {1: 0.9, 2: 0.8, ...}

{0: 1.0, 189: 0.9901179671287537, 3080: 0.9858693480491638, 6194: 0.9849436283111572, 3120: 0.9836863875389099}


In [10]:
body = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
        ],
    }
]

prediction = model.predict(body)[0]

In [11]:
user_idx = 0
query_vector = np.array(prediction["item_vector"])
result = search(query_vector, item_index, top_k=30)

item_indies = list(result.keys())[1:]

recommendation = {}
for i in item_indies:
    item_id = item_id_maps[i]
    recommendation[item_id] = result[i]

recommendation

{3719: 0.9747278690338135,
 6383: 0.9745081663131714,
 6865: 0.974403977394104,
 8458: 0.973960816860199,
 9504: 0.9739301204681396,
 4194: 0.9738999009132385,
 9568: 0.9734377861022949,
 7528: 0.973332405090332,
 3891: 0.973092257976532,
 8033: 0.9730600118637085,
 5582: 0.9730135202407837,
 7091: 0.9730099439620972,
 590: 0.972985029220581,
 8794: 0.972968578338623,
 9107: 0.9728628993034363,
 6990: 0.9728279113769531,
 2743: 0.9727413058280945,
 1056: 0.9726566076278687,
 3715: 0.9725311994552612,
 1401: 0.9724861979484558,
 7725: 0.9724794030189514,
 5521: 0.9724524617195129,
 7601: 0.9724494814872742,
 1835: 0.9723831415176392,
 5638: 0.9723823666572571,
 2895: 0.9723414778709412,
 8364: 0.9723048210144043,
 4086: 0.9722937941551208,
 2808: 0.9722592234611511}

In [12]:
df_recommendation = df_item[
    df_item["item_id"].isin(list(recommendation.keys()))
].reset_index(drop=True)

df_recommendation

,item_id,title,color,style,fit,material,season,sleeve,category
0,590,가을 감성 슬림핏 스웨이드 니트,스카이블루,스트리트,슬림핏,스웨이드,가을,롱슬리브,니트
1,1056,가을 감성 루즈핏 폴리 니트,아이보리,포멀,루즈핏,폴리,가을,숏,니트
2,1401,봄 감성 루즈핏 메쉬 니트,네이비,모던,루즈핏,메쉬,봄,롱,니트
3,1835,봄 감성 슬림핏 모달 재킷,카키,러블리,슬림핏,모달,봄,롱슬리브,재킷
4,2743,여름 감성 루즈핏 메쉬 슬랙스,라벤더,러블리,루즈핏,메쉬,여름,숏슬리브,슬랙스
5,2808,겨울 감성 오버핏 스웨이드 스커트,오렌지,미니멀,오버핏,스웨이드,겨울,민소매,스커트
6,2895,가을 감성 루즈핏 플리스 니트,라벤더,캐주얼,루즈핏,플리스,가을,숏슬리브,니트
7,3715,겨울 오버핏 레드 모달 롱 스커트,레드,포멀,오버핏,모달,겨울,롱,스커트
8,3719,가을 감성 루즈핏 스웨이드 슬랙스,블랙,포멀,루즈핏,스웨이드,가을,롱슬리브,슬랙스
9,3891,가을 감성 슬림핏 모달 원피스,블랙,댄디,슬림핏,모달,가을,7부,원피스
